# Assignment-Adapter Demo
In this we look at another builtin adapter, the Simple Assignment Adapter `enbios/base/adapters_aggregators/builtin/assignment_adapter.py`. This adapter does not any specific calculations. Instead, it allows the user to introduce fixed values, that should come from some external source into the enbios tree calculation. This includes not just the outputs of structural nodes, but in particular their impact results.

These values can be either in the experiment configuration file or for convenience in a referenced csv file. The values can be specified in such a way, that scenario outputs and result values have consistent and valid units.

In addition, compared to the brightway adapter, this adapter allows to specify _multiple outputs_ for each node, and also nodes that do not specify any output, even tho they have some impacts. 

The adapters dedicated name is `assignment-adapter` tho for nodes, the indicator `assign` can also be used.

We will start with a minimal experiment configuration, that includes one node with a simple assignment 

In [1]:
import pandas as pd

from enbios import Experiment

simple_assignment_node = {
    "name": "simple_node",
    "adapter": "assign",
    "config": {
        "outputs": [{"unit": "l"}],
        "default_outputs": [{"magnitude": 4}],
        "default_impacts": {"co2": {"unit": "kg", "magnitude": 10}},
    },
}

experiment_data = {
    "adapters": [
        {
            "adapter_name": "assignment-adapter",
            # by default this adapter needs no specific config
            "methods": {"co2": "kg"},
        }
    ],
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [simple_assignment_node],
    },
}

experiment = Experiment(experiment_data)
experiment.run()

The config of an assign node is structured the following way:
These are the fields that can/must be set:
```
    node_name: a unique string
    outputs: a list that specifies the outputs
    default_outputs: a list of default-outputs (optional) 
    default_impacts: a dictionary of default impacts
```

In detail:
__outputs:__ each output in that list contains the fields: `unit` and `label` (optional). The unit specifies the unit for that output. When default or scenario outputs do not specify any unit, this one will be used, and if they do, it must be compatible with this unit, e.g. W, kW, MW, TW, ...

The label is used in order to indicate if nodes outputs should be merged  or not(by the sum-aggregator). By default, outputs that do not have any label get merged, when their units are compatible with each other. If outputs have labels, they will only be merged with other nodes outputs, when they have the same label. Further down, we will see this in an example.

__default_outputs:__ Default outputs are used in scenarios, when there is no output specified. A default output (one item in the list) can also be `None` (`null` in json), meaning an output always needs to be defined. The unit can be omited, in that case the normal output unit will be used (defined in `outputs`).

__default_impacts:__ The default impacts, which will be used, when some impact is not specified in a scenario, is a dictionary that maps from a method (as specified in the config of the adapter) to a value (`unit` and `magnitude`)


In this example, we see how a node can be specified, when there are scenarios, which define specific outputsw and impacts. The two fields for a node in a scenario are `outputs` and `impacts` and the structure is the same as for default values.

In [2]:
from enbios import Experiment
from enbios.base.adapters_aggregators.builtin.assignment_adapter import AssignmentAdapter

simple_assignment_node = {
    "name": "simple_node",
    "adapter": "assign",
    "config": {"outputs": [{"unit": "l"}]},
}

experiment_data = {
    "adapters": [
        {
            "adapter_name": "assignment-adapter",
            # by default this adapter needs no specific config
            "methods": {"co2": "kg"},
        }
    ],
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [simple_assignment_node],
    },
    "scenarios": [
        {
            "name": "scenario 1",
            "nodes": {
                "simple_node": {
                    "outputs": [{"magnitude": 4}],
                    "impacts": {"co2": {"unit": "kg", "magnitude": 10}},
                }
            },
        }
    ],
}

experiment = Experiment(experiment_data)
experiment.run()

In [3]:
experiment.adapters[0].nodes

In [4]:
AssignmentAdapter.get_config_schemas()

## Output merging with labels

In this example we create 2 nodes, the first one with 4 outputs and the 2nd one with 2 like so:

__1. node__
- l (liter)
- h (hours)
- h (hours), label:labor
- kW (kilo-Watt)

__2. node__
- h (hours)
- kW (kilo-Watt)

We then just look at the outputs of the root node, which merges the outputs of the 2 nodes.
The root node will merge the two hour outputs, which have no labels, and the kW outputs (also without labels), but leave the hour output of the 1. node, which has no label, separate.

In [5]:
experiment_data = {
    "adapters": [
        {
            "adapter_name": "assignment-adapter",
            # by default this adapter needs no specific config
            "methods": {"co2": "kg"},
        }
    ],
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [
            {
                "name": "node1",
                "adapter": "assign",
                "config": {
                    "outputs": [
                        {"unit": "l"},
                        {"unit": "h"},
                        {"unit": "h", "label": "labor"},
                        {"unit": "kW"},
                    ],
                    "default_outputs": [
                        {"magnitude": 5},
                        {"magnitude": 1},
                        {"magnitude": 4},
                        {"magnitude": 75},
                    ],
                    "default_impacts": {"co2": {"unit": "kg", "magnitude": 10}},
                },
            },
            {
                "name": "node2",
                "adapter": "assign",
                "config": {
                    "outputs": [{"unit": "h"}, {"unit": "kW"}],
                    "default_outputs": [{"magnitude": 1}, {"magnitude": 10}],
                    "default_impacts": {"co2": {"unit": "kg", "magnitude": 10}},
                },
            },
        ],
    },
}

experiment = Experiment(experiment_data)
res = experiment.run()
res["default scenario"]["output"]

## Using csv files

Next, we demonstrate how to use csv files to specify the outputs and impacts for assignment adapter nodes. Instead of specifying the outputs and impacts directly in the config, we can specify one csv file, which contains the outputs and impacts for all assignment nodes in our experiment, their default values but also their scenario values.

Let's look at the structure of the csv file.
A minimal csv file looks like this, where we only specify default value for a single scenario:


| node_name | outputs_1_unit | default_outputs_1_unit | default_outputs_1_magnitude | default_impacts_co2_unit | default_impacts_co2_magnitude |



```
node_name
output_{id}_unit
output_{id}_label
default_outputs_{id}_unit
default_outputs_{id}_magnitude 
default_impacts_{impact_id}_unit
default_impacts_{impact_id}_magnitude
```
Since we can have multiple outputs, we need to specify the `id` of each output in this csv file. These ids have no further purpose other than linking different columns in the csv file. An id can be any string of characters and numbers. The `ìmpact_id`s must match the method names defined in the adapter.

In [6]:
pd.read_csv("data/assigment_csv_files/assignment1.csv")

Note, how the `default_outputs_{id}_unit` is as with the json format only optional. If it is not specified, the unit from the `outputs` field will be used.

It is also important that we still need to the nodes in the hierarchy, with their name and adapter, but we can leave out the config.

In [7]:
experiment_data = {
    "adapters": [
        {
            "adapter_name": "assignment-adapter",
            "methods": {"co2": "kg"},
            "config": {"source_csv_file": "data/assigment_csv_files/assignment1.csv"},
        }
    ],
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [{"name": "n1", "adapter": "assign"}],
    },
}

from enbios import Experiment

experiment = Experiment(experiment_data)
experiment.run()

Default values should only be defined in the first row of a new node. The other rows are for scenario values, one row per scenario.

Similar to the nodes, the scenarios must be defined in the `scenarios` field of the experiment.

In [8]:
pd.read_csv("data/assigment_csv_files/assignment2.csv").fillna("")

In [9]:
experiment_data["adapters"][0]["config"][
    "source_csv_file"
] = "data/assigment_csv_files/assignment2.csv"
experiment_data["scenarios"] = [{"name": "sc1"}, {"name": "sc2"}]
experiment = Experiment(experiment_data)
experiment.run()

In [10]:
# Finally a mixed example
pd.read_csv("data/assigment_csv_files/assignment3.csv").fillna("")

In [11]:
import json

experiment_data["adapters"][0]["config"][
    "source_csv_file"
] = "data/assigment_csv_files/assignment3.csv"
experiment_data["scenarios"] = [{"name": "sc1"}, {"name": "sc2"}]
experiment_data["hierarchy"]["children"] = [
    {"name": "n1", "adapter": "assign"},
    {"name": "n2", "adapter": "assign"},
]
experiment = Experiment(experiment_data)
print(json.dumps(experiment.run(), indent=2))